# Global Warming Level Timing Tools

This notebook is designed to explore the estimated timing of Global Warming Levels (GWLs) based on the IPCC's warming trajectory estimates from the AR6 report



In [15]:
from climakitae.core.paths import (
    ssp119_file,
    ssp126_file,
    ssp245_file,
    ssp370_file,
    ssp585_file,
    hist_file,
)
import pandas as pd
import numpy as np

from climakitae.util.utils import read_csv_file

ssp119_data = read_csv_file(ssp119_file, index_col="Year")
ssp126_data = read_csv_file(ssp126_file, index_col="Year")
ssp245_data = read_csv_file(ssp245_file, index_col="Year")
ssp370_data = read_csv_file(ssp370_file, index_col="Year")
ssp585_data = read_csv_file(ssp585_file, index_col="Year")
hist_data = read_csv_file(hist_file, index_col="Year")

ssp_dict = {
    "Historical": hist_data,
        "SSP 1-1.9": ssp119_data,
        "SSP 1-2.6": ssp126_data,
        "SSP 2-4.5": ssp245_data,
        "SSP 3-7.0": ssp370_data,
        "SSP 5-8.5": ssp585_data,
    }


In [ ]:
def get_gwl_at_year(year, ssp='all'):

    wl_timing_df = pd.DataFrame(columns=["5%", "Mean", "95%"])

    if year >= 2015:
        if ssp== 'all':
            ssp_list = ['SSP 1-1.9', 'SSP 1-2.6', 'SSP 2-4.5', 'SSP 3-7.0', 'SSP 5-8.5']
        else:
            ssp_list = [ssp]

        for ssp in ssp_list:

            # get the data for the ssp
            ssp_data = ssp_dict[ssp]
            # get the data for the year
            # get the data for the year
            if year not in ssp_data.index:
                print(f"Year {year} not found in {ssp}")
                wl_timing_df.loc[ssp] = [np.nan, np.nan, np.nan]
                continue
            # get the data for the year
            ssp_data = ssp_data.loc[year]
            wl_timing_df.loc[ssp] = round(ssp_data,2)
    
    else:
        # get data from historical for years before 2015
        if ssp != 'all':
            print(f"Year {year} before 2015, using Historical data")
        hist_data = ssp_dict['Historical']
        # get the data for the year
        if year not in hist_data.index:
            print(f"Year {year} not found in Historical")
            wl_timing_df.loc['Historical'] = [np.nan, np.nan, np.nan]
            return wl_timing_df
        hist_data = hist_data.loc[year]
        wl_timing_df.loc['Historical'] = round(hist_data,2)
    return wl_timing_df




In [18]:
get_gwl_at_year(2018)

,5%,Mean,95%
SSP 1-1.9,1.07,1.18,1.29
SSP 1-2.6,1.06,1.18,1.30
SSP 2-4.5,1.06,1.18,1.30
SSP 3-7.0,1.06,1.17,1.29
SSP 5-8.5,1.08,1.20,1.33


In [13]:
get_gwl_at_year(2032)

,5%,Mean,95%
SSP 1-1.9,1.24,1.47,1.73
SSP 1-2.6,1.27,1.50,1.77
SSP 2-4.5,1.31,1.53,1.79
SSP 3-7.0,1.32,1.55,1.81
SSP 5-8.5,1.39,1.64,1.93


In [6]:
get_gwl_at_year(2031)

,5%,Mean,95%
SSP 1-1.9,1.23,1.46,1.70
SSP 1-2.6,1.26,1.48,1.74
SSP 2-4.5,1.29,1.51,1.75
SSP 3-7.0,1.30,1.52,1.77
SSP 5-8.5,1.36,1.61,1.89


In [13]:
get_gwl_at_year(2050)

,5%,Mean,95%
SSP 1-1.9,1.26,1.57,1.92
SSP 1-2.6,1.39,1.72,2.16
SSP 2-4.5,1.61,1.95,2.40
SSP 3-7.0,1.74,2.10,2.56
SSP 5-8.5,1.90,2.35,2.87


In [131]:
get_gwl_at_year(2073, 'SSP 3-7.0')

,5%,Mean,95%
SSP 3-7.0,2.327845,2.912322,3.642675


In [19]:
def get_year_at_gwl(gwl, ssp='all'):
    wl_timing_df = pd.DataFrame(columns=["5%", "Mean", "95%"])

    if ssp== 'all':
        ssp_list = ['Historical','SSP 1-1.9', 'SSP 1-2.6', 'SSP 2-4.5', 'SSP 3-7.0', 'SSP 5-8.5']
    else:
        ssp_list = [ssp]
    
    for ssp in ssp_list:

        if ssp == "Historical":
            ssp_start = 1950
        else:
            ssp_start = 2015

        ssp_selected = ssp_dict[ssp]  # data selected

        # Only add data for a scenario if the mean and upper bound of uncertainty reach the gwl
        if (np.argmax(ssp_selected["Mean"] > gwl) > 0) and (
            (np.argmax(ssp_selected["95%"] > gwl)) > 0
        ):

            x_5 = int(ssp_start + np.argmax(ssp_selected["95%"] > gwl))

            if np.argmax(ssp_selected["5%"] > gwl):
                x_95 = int(ssp_start + np.argmax(ssp_selected["5%"] > gwl))
            else:
                # set year to 2100+ if the lower bound of uncertainty does not hit the gwl
                x_95 = np.nan

            year_gwl_reached = (
                ssp_selected.where(ssp_selected["Mean"] > gwl)
                .dropna()
                .index[0]
            )

        else:
            x_95 = np.nan
            x_5 = np.nan
            year_gwl_reached = np.nan

        wl_timing_df.loc[ssp] = [x_5, year_gwl_reached, x_95]

    return wl_timing_df

In [35]:
get_year_at_gwl(1.1111)

,5%,Mean,95%
Historical,NaN,NaN,NaN
SSP 1-1.9,NaN,NaN,NaN
SSP 1-2.6,NaN,NaN,NaN
SSP 2-4.5,NaN,NaN,NaN
SSP 3-7.0,NaN,NaN,NaN
SSP 5-8.5,NaN,NaN,NaN


In [122]:
get_year_at_gwl(2.1)

,5%,Mean,95%
Historical,NaN,NaN,NaN
SSP 1-1.9,NaN,NaN,NaN
SSP 1-2.6,NaN,NaN,NaN
SSP 2-4.5,2042.0,2057.0,2092.0
SSP 3-7.0,2040.0,2050.0,2065.0
SSP 5-8.5,2036.0,2045.0,2057.0


In [134]:
get_year_at_gwl(3.2, 'SSP 5-8.5')

,5%,Mean,95%
SSP 5-8.5,2056.0,2069.0,2089.0
